## CVEP pipeline

---
By: Jorge Sanmartin Martinez (s1150007), Paul Verhoeven (s1086755), Radovan Vodila (s1143934), and Thomas Kooiman (s1018394)

## 0. handle imports

In [20]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

import h5py
import mne

## 1. Load the dataset

In [38]:
dataset_folder = Path("dataset") / "sub-01" / "block_1"



codes = dataset_folder / "mgold_61_6521.npz"
dataset = dataset_folder.glob("*.mat")

gdf_file = dataset_folder / "sub-01_20181128_block_1_main_eeg.gdf"

raw=mne.io.read_raw_gdf(gdf_file)
print(raw.info)
raw.to_data_frame()




Extracting EDF parameters from c:\Users\paul_\Documents\school\bci\aBCI3.6\dataset\sub-01\block_1\sub-01_20181128_block_1_main_eeg.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
AF3, F3, FC5, P7, P8, FC6, F4, AF4
Creating raw.info structure...
<Info | 8 non-empty values
 bads: []
 ch_names: STATUS, AF3, F3, FC5, P7, P8, FC6, F4, AF4
 chs: 1 Stimulus, 8 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 256.0 Hz
 meas_date: unspecified
 nchan: 9
 projs: []
 sfreq: 512.0 Hz
 subject_info: 2 items (dict)
>


,time,STATUS,AF3,F3,FC5,P7,P8,FC6,F4,AF4
0,0.000000,0.0,-27.438843,-3.892944,63.916382,59.438843,18.959961,34.417725,114.649902,33.656738
1,0.001953,0.0,-2119.980225,-300.325317,4934.645508,4588.413086,1463.078735,2657.300781,8852.711914,2600.531494
2,0.003906,0.0,-2866.390137,-407.768921,6665.097168,6196.198730,1976.392456,3592.297607,11955.258789,3512.349609
3,0.005859,0.0,-2628.119141,-373.761597,6114.492676,5684.946777,1814.671265,3296.684814,10964.780273,3222.350342
4,0.007812,0.0,-2715.971436,-379.893799,6316.330078,5873.454102,1875.583252,3404.657227,11328.343750,3331.691895
...,...,...,...,...,...,...,...,...,...,...
341474,666.941406,0.0,-4430.994629,-1132.488892,4783.828125,5038.476074,1515.405029,3148.337158,10314.791992,941.644897
341475,666.943359,0.0,-4433.126465,-1144.315308,4784.632812,5035.869141,1516.374268,3148.362793,10313.591797,937.338745
341476,666.945312,0.0,-4438.213867,-1139.248291,4783.097168,5034.797852,1514.560669,3145.760498,10309.438477,940.605469
341477,666.947266,0.0,-4436.717285,-1123.964478,4781.784180,5035.194824,1512.245728,3143.502197,10307.357422,931.008301


In [ ]:
# load the associated label files
trainlabels_file = dataset_folder /"trainlabels.mat"

file = h5py.File(trainlabels_file, "r")


with h5py.File(trainlabels_file, 'r') as f:
    v_data = f['v'][:]  # Load the dataset
    print(v_data.shape)
    print(v_data)

Type of 'v': <class 'h5py._hl.dataset.Dataset'>
'v' is a dataset. Shape and dtype:
(20, 1) float64
(20, 1)
[[ 5.]
 [17.]
 [18.]
 [11.]
 [ 8.]
 [ 9.]
 [ 3.]
 [ 7.]
 [19.]
 [10.]
 [ 6.]
 [13.]
 [20.]
 [ 4.]
 [12.]
 [16.]
 [14.]
 [15.]
 [ 2.]
 [ 1.]]
